In [1]:

import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate

# Get the absolute path of the parent directory of the current file (which is 'test')
# current_dir = os.getcwd()
# model_path = os.path.join(current_dir, '..', 'model')
# sys.path.append(model_path)
sys.path.append('..')

from model.Force import HullForce, ControlForce, TowingForce
from model.RigidBody import RigidBody
from model.Simulation import Simulation, Simulation_Result

In [2]:
# Set parameters
m = 0.877 # kg
Vol = 733159.45810955 * 10 ** (-9) #m3
Iyy = 6610.17475482 * 10 ** (-6) * 10 ** (3) #kgm2
hull_surface_area = 0.0088 #m2
hull_cd = 1.2
hull_corr_factor = 0.75

r_xi = -111.76 * 10 ** (-3) #m 
r_zi = 0.722 * 10 ** (-3) #m

r_xh = 0
r_zh = 0

r_xb = 0
r_zb = 0

r_xt = 42.9 * 10**(-3)
r_zt = -75 * 10**(-3)

rho = 1000 #kg/m3

# Aero foil parameters
AR = 2.61 #aspect ratio
area = 0.025 #Surface area
C_L_a = 0.1/np.deg2rad(1)
C_L_alpha_offset = 0.5
C_D_0 = 1.328/np.sqrt(200000)

print(C_L_a)

5.729577951308232


In [3]:
rigidbody = RigidBody(m, Vol, Iyy, np.array([r_xb, r_zb]))
towForce = TowingForce(np.array([r_xt,r_zt]),10,np.deg2rad(30))
controlForce = ControlForce(np.array([r_xi,r_zi]),np.deg2rad(-5),AR,area,C_L_a,C_L_alpha_offset,C_D_0)
hullForce = HullForce(hull_surface_area,np.array([r_xh,r_zh]),hull_cd,hull_corr_factor)

sim = Simulation(rigidbody,towForce,hullForce,[controlForce])


In [4]:
results_LS = sim.solve_equilibrium_state_LS(2)

Optimization Results:
----------------------
Parameter       Value           Units     
----------------------
Pitch Angle     -3.68           degrees   
Delta_t         27.05           degrees   
Towing Force    18.62           N         
Delta_i         3.94            degrees   
----------------------
Fx:             -0.00          N         
Fz:             0.00           N         
My:             -0.00          Nm        
Residual Norm: 0.000000
Optimization successful!


In [5]:
print(np.rad2deg(sim.eq_sim.pitch_angle[0]))


-3.6793246949026313


In [8]:
    
def plotfbd(sim:object):
# Define the rotation angle in radians
    theta = -sim.pitch_angle[0] # Rotate by pitch (adjust as needed)
    # Rotation matrix
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ])

    # Define forces with their points of application (relative to origin)
    forces = [
        {"point": (r_xi, r_zi), "vector": (0, sim.sim.control_force_z)},            # Control z-force
        {"point": (r_xi, r_zi), "vector": (sim.sim.control_force_x, 0)},            # Control x-force
        {"point": (r_xh, r_zh), "vector": (sim.sim.hull_force_x, sim.sim.hull_force_z)}, # Hull drag
        {"point": (r_xt, r_zt), "vector": (sim.sim.tow_force_x, sim.sim.tow_force_z)},   # Tow force
        {"point": (r_xb, r_zb), "vector": (sim.sim.buoyancy_force_x, sim.sim.buoyancy_force_z)}, # Buoyancy
        {"point": (0, 0), "vector": (sim.sim.mass_force_x, sim.sim.mass_force_z)}       # Weight
    ]

    # Determine the maximum length of the vectors
    max_length = max(np.linalg.norm(force['vector']) for force in forces)

    # Specify the desired maximum length on the plot
    desired_max_length = 0.1

    # Calculate scaling factor
    scaling_factor = desired_max_length / max_length if max_length > 0 else 1

    # Define rectangle properties
    rect_center = (0,0)  # Center of the rectangle
    rect_width, rect_height = 0.15,.08

    # Define rectangle corners (before rotation)
    corners = [
        (rect_center[0] - rect_width / 2, rect_center[1] - rect_height / 2),  # Bottom-left
        (rect_center[0] + rect_width / 2, rect_center[1] - rect_height / 2),  # Bottom-right
        (rect_center[0] + rect_width / 2, rect_center[1] + rect_height / 2),  # Top-right
        (rect_center[0] - rect_width / 2, rect_center[1] + rect_height / 2)   # Top-left
    ]

    # Rotate forces and rectangle corners
    transformed_forces = []
    for force in forces:
        # Transform point and vector
        rotated_point = np.dot(rotation_matrix, force["point"])
        rotated_vector = np.dot(rotation_matrix, force["vector"])

        # Scale the rotated vector
        scaled_vector = rotated_vector * scaling_factor
        
        # Store transformed point and vector
        transformed_forces.append({"point": rotated_point, "vector": scaled_vector})

    # Rotate rectangle corners
    rotated_corners = [np.dot(rotation_matrix, corner) for corner in corners]
    rotated_corners.append(rotated_corners[0])  # Close the rectangle by repeating the first point

    # Calculate limits based on rotated forces and rectangle
    padding = 0.05
    x_min = min(point[0] + vector[0] for force in transformed_forces for point, vector in zip([force['point']], [force['vector']])) - padding
    x_max = max(point[0] + vector[0] for force in transformed_forces for point, vector in zip([force['point']], [force['vector']])) + padding
    y_min = min(point[1] + vector[1] for force in transformed_forces for point, vector in zip([force['point']], [force['vector']])) - padding
    y_max = max(point[1] + vector[1] for force in transformed_forces for point, vector in zip([force['point']], [force['vector']])) + padding

    # Set up plot
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_aspect('equal')

    # Set axis limits
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    # Plot transformed forces
    for force in transformed_forces:
        point = force["point"]
        vector = force["vector"]
        ax.quiver(point[0], point[1], vector[0], vector[1], angles='xy', scale_units='xy', scale=1, color='blue')
        ax.plot([0,point[0]], [0,point[1]],'.-')

    # Plot the rotated rectangle
    rotated_x = [corner[0] for corner in rotated_corners]
    rotated_y = [corner[1] for corner in rotated_corners]
    ax.plot(rotated_x, rotated_y, 'g-')  # Green rectangle outline

    # Flip the appearance of the axes without changing vector directions
    ax.invert_xaxis()  # Flip x-axis appearance
    ax.invert_yaxis()  # Flip y-axis appearance

    # Optional: Add grid, labels, and title
    plt.grid(True)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Free Body Diagram with Rotated Forces and Rectangle')

    plt.show()

In [10]:
plotfbd(sim.sim)

IndexError: list index out of range

In [ ]:
velocity=2
pitch_angle = results_LS[0]
jacobian = sim.calculate_jacobian(velocity, pitch_angle, 1e-5)
eigs= np.linalg.eig(jacobian)




pstate [ 1.00000000e-05  0.00000000e+00 -3.49441006e-02  2.00000000e+00
  0.00000000e+00  0.00000000e+00]
xdot pertu [ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -1.18007710e-11
  2.99824667e-10 -3.57090117e-10]
pstate [ 0.00000000e+00  1.00000000e-05 -3.49441006e-02  2.00000000e+00
  0.00000000e+00  0.00000000e+00]
xdot pertu [ 2.00000000e+00  0.00000000e+00  0.00000000e+00 -1.18007710e-11
  2.99824667e-10 -3.57090117e-10]
pstate [ 0.         0.        -0.0349341  2.         0.         0.       ]
xdot pertu [ 2.00000000e+00  0.00000000e+00  0.00000000e+00  8.12340129e-05
  3.35359872e-06 -2.05925412e-06]
pstate [ 0.         0.        -0.0349441  2.00001    0.         0.       ]
xdot pertu [ 2.00001000e+00  0.00000000e+00  0.00000000e+00 -1.18007710e-11
  2.99824667e-10 -3.57090117e-10]
pstate [ 0.00000000e+00  0.00000000e+00 -3.49441006e-02  2.00000000e+00
  1.00000000e-05  0.00000000e+00]
xdot pertu [ 2.00000000e+00  1.00000000e-05  0.00000000e+00 -1.18007710e-11
  2.99824667e-10

In [ ]:
print(eigs[0])

[0.+0.j         0.+0.j         0.+0.j         0.+0.j
 0.+0.45375071j 0.-0.45375071j]
